# Power Calculations for Planning Program Evaluations
Bryan Graham
University of California - Berkeley  

January 2019
<br>
<br>
#### Code citation:
<br>
Graham, Bryan S. (2019). "Power Calculations for Planning Program Evaluations Python Jupyter Notebook," (Version 1.0) [Computer program]. Available at http://bryangraham.github.io/econometrics/ (Accessed 27 May 2019)    
<br>
<br>
Begin by loading the *ipt* module. For information about this module, including how to install it, please see this blog [post](http://bryangraham.github.io/econometrics/causal/inference/2016/05/15/IPT-module.html).    
<br>
<br>
We will also need several standard libraries that are all part of the usual Python "scientific stack".

In [1]:
# Append location of ipt module base directory to system path
# NOTE: only required if permanent install not made (see link above)
import sys
sys.path.append('/Users/bgraham/Dropbox/Sites/software/ipt/')

# Load ipt module
import ipt as ipt

In [2]:
# Direct Python to plot all figures inline (i.e., not in a separate window)
%matplotlib inline

# Load libraries
import numpy as np
import scipy as sp

import pandas as pd

Adjust the working directory assignment below to where you have saved the divers.pkl file on your local machine. This file is a pickled version of a pandas dataframe. This dataframe include information on lobster/conch diving participation and village of residence for 1,078 adult male Miskitos living in one of 60 villages in the Gracias a Dios region of Honduras. Also included are a set of sampling weights, but these do not feature in the power analysis below for reasons I will explain.

In [3]:
# Working (root) directory
workdir =  '/Users/bgraham/Dropbox/Teaching/Berkeley_Courses/Ec141/Spring2019/'

Read in dataframe and compute some summary statistics. We can observe that 26.5 percent of the 1,078 individuals in the dataset were active divers in the 2002/3 season or were divers sometime in the past.

In [4]:
# Read in La Moskitia 2003 diver survey extract
divers = pd.read_pickle(workdir + "Datasets/divers.pkl") 
print(divers.describe())

             diver           sw  constant
count  1078.000000  1078.000000    1078.0
mean      0.265306    14.275484       1.0
std       0.441701     6.084577       0.0
min       0.000000     2.115400       1.0
25%       0.000000    10.747700       1.0
50%       0.000000    16.086900       1.0
75%       1.000000    20.108600       1.0
max       1.000000    22.947400       1.0


The next snippet of code uses the **ipt** module's implementation of OLS to compute an estimate of the mean participation rate in the lobster/conch diving industry among adult miskito males residing in Gracias a Dios. We could alternatively use statsmodels OLS command, but I find my own _ipt_ implementation better tailored to settings where survey weights and clustered standard errors are required routinely.
<br>
<br>
Although 27 percent of our sample are current or past divers, once we taking into account the stratified nature of the survey from which our sample was drawn, we produce an estimate of the population rate of participation of just 16 percent. With a 95 percent confidence interval of (roughly) between 11 and 20 percent. We will nevertheless base our analysis upon the unweighted data. The reason for doing so is the belief that, while the sixty villages included in our sample may not be perfectly representative of La Moskitia, they are representative of the villages where our program is likely to operate. This is consistent with the original purposes of the survey, which was to learn more about lobster divers and their families in the region. Hence it purposively over-sampled villages where diving was thought to be common. These are precisely those villages that are well suited to host the hypothetical intervention.

In [5]:
# Estimate adult Miskito male participation rate in Lobster diving
Y = divers['diver']
X = divers['constant'].to_frame()

v_id = pd.factorize(divers['village'])[0]
v_id = pd.Series(v_id, name = 'village')

[beta_hat, vcov_hat ,_ ,_ ,_] = ipt.ols(Y, X, c_id=v_id, s_wgt=divers['sw'], nocons=True, silent=False)


-----------------------------------------------------------------------
-                     OLS ESTIMATION RESULTS                          -
-----------------------------------------------------------------------
Dependent variable:        diver
Number of observations, n: 1078



Independent variable       Coef.    ( Std. Err.)     (0.95 Confid. Interval )
-------------------------------------------------------------------------------------------
constant                   0.157673 (  0.022852)     (  0.112883 ,  0.202463)

-------------------------------------------------------------------------------------------
NOTE: Cluster-robust standard errors reported
      Cluster-variable   = village
      Number of clusters = 60
NOTE: (Sampling) Weighted estimates computed.
      Weight-variable   = sw


/Users/bgraham/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:51: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


## Power calculation for hypothetical Miskito diver intervention
To perform the power calculations for the hypothetical intervention described in _Lecture Note 3: Power Calculations_ I begin with a simple one-way ANOVA analysis. I use the output from this analysis to compute an estimate of the variance of diver participation as well as the intra-cluster correlation. The binary outcome is whether the individual dives for lobster/conch or not and the factors correspond to villages of residence.
<br>
<br>
The statsmodels module has a servicable implementation of ANOVA which I use here. Unfortunately it does not directly report an estimate of intra-cluster correlation, but it is easy to construct one from the information in the ANOVA table. I use the $ \omega^{2} $ estimate of intra-cluster correlation, because it is less biased than the more common $ \eta^{2} $ measure. In the present case the difference between the two measures is meaningful. They imply substantially different minimum sample sizes to achieve the same level of power.

In [6]:
# load required libraries from statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols as sm_ols

# Fit linear model with village dummies
divers_anova = sm_ols('diver ~ C(village)', data=divers).fit()      # Fit linear model

# Construct one-way ANOVA table
table        = sm.stats.anova_lm(divers_anova, typ=1)               # Anova table

# Extra table elements to compute omega_sq (a nearly unbiased estimate of rho)
SS_b         = table['sum_sq'][0]                                   # Between-village sum-of-squares
SS_w         = table['sum_sq'][1]                                   # Within-village sum-of-squares
df_b         = table['df'][0]                                       # Between degress of freedom 
df_w         = table['df'][1]                                       # Within degress of freedom 
omega_sq     = (SS_b - (SS_w/df_w)*df_b)/ \
               (SS_b + SS_w + (SS_w/df_w))                          # rho estimate (omega_sq)
eta_sq       = SS_b / (SS_b + SS_w )                                # rho estimate (eta_sq)   

# Calculate the overall standard deviation of the outcome
sigma = np.sqrt((SS_b + SS_w + (SS_w/df_w))/len(divers['diver']))
Q     = divers['diver'].mean()                                      # Sample participation rate
                                                                    # NOTE: sigma2 = Q(1-Q)
# Print out ANOVA results
print("Oneway ANOVA table for Lobster Diving Outcome")
print("----------------------------------------------------")
print("")
print(table)
print("")
print("----------------------------------------------------")
print("Pr(Diver=1) = " + "%0.4f" % Q)
print("sigma       = " + "%0.4f" % sigma)
print("omega_sq    = " + "%0.4f" % omega_sq)
print("eta_sq      = " + "%0.4f" % eta_sq)

Oneway ANOVA table for Lobster Diving Outcome
----------------------------------------------------

                df      sum_sq   mean_sq         F        PR(>F)
C(village)    59.0   40.126416  0.680109  4.072746  1.142984e-20
Residual    1018.0  169.996033  0.166990       NaN           NaN

----------------------------------------------------
Pr(Diver=1) = 0.2653
sigma       = 0.4417
omega_sq    = 0.1440
eta_sq      = 0.1910


With the above information from the baseline survey calculated I can now proceed to calculate minimum sample sizes needed to reliabily (i.e., $ \beta = 0.80 $) detect effects of different sizes. I focus on sampling plans which target $T = 20$ respondents per village (which is about what is available in the baseline survey).
<br>
<br>
The calculations are based on the minimum sample size formulae given in _Lecture Note 3: Power Calculations_. For the two-sided test case I use the approximation which appears in Dulfo et al (2007, _Handbook of Development Economics_).
<br>
<br>
Note in what follows I actually solve for $N_{1}=\lambda N$, and round this _up_ to the nearest integer. With $\lambda = 0.5 $, as is optimal under the homogenous variance assumption, this ensures that an even number of clusters is chosen...which can then we divided equally into treatmend and controls.

In [7]:
alpha  = 0.05
power  = [0.8]
lmbda  = 0.5
mde    = [0.05, 0.10, 0.15]
rho    = [0, omega_sq, eta_sq]

T     = 20

print("")                
print("One-sided Test")
for beta in power:
    for theta in mde:
        for p in rho:
            N1 = (p + (1-p)/T)*((1-lmbda)**-1)*((sigma/theta)**2) \
                             *(sp.stats.norm.ppf(1-alpha) - sp.stats.norm.ppf(1-beta))**2
            N  = np.ceil(((1-lmbda)/lmbda)*N1) + np.ceil(N1)   
            print("beta = "  + "%0.2f" % beta + ", mde = " + "%0.2f" % theta + ", sigma  = " + "%0.3f" % sigma   \
                + ", rho = " + "%0.3f" % p    + ", N = "   + "%0.0f" % N     + ", n = NT = " + "%0.0f" % (N*T))             
                
print("")                
print("Two-sided Test (approximate)")
for beta in power:
    for theta in mde:
        for p in rho:
            N1 = (p + (1-p)/T)*((1-lmbda)**-1)*((sigma/theta)**2) \
                              *(sp.stats.norm.ppf(1-alpha/2) - sp.stats.norm.ppf(1-beta))**2
            N  = np.ceil(((1-lmbda)/lmbda)*N1) + np.ceil(N1)          
            print("beta = "  + "%0.2f" % beta + ", mde = " + "%0.2f" % theta + ", sigma  = " + "%0.3f" % sigma  \
                + ", rho = " + "%0.3f" % p    + ", N = "   + "%0.0f" % N     + ", n = NT = "  + "%0.0f" % (N*T))             


One-sided Test
beta = 0.80, mde = 0.05, sigma  = 0.442, rho = 0.000, N = 98, n = NT = 1960
beta = 0.80, mde = 0.05, sigma  = 0.442, rho = 0.144, N = 362, n = NT = 7240
beta = 0.80, mde = 0.05, sigma  = 0.442, rho = 0.191, N = 448, n = NT = 8960
beta = 0.80, mde = 0.10, sigma  = 0.442, rho = 0.000, N = 26, n = NT = 520
beta = 0.80, mde = 0.10, sigma  = 0.442, rho = 0.144, N = 92, n = NT = 1840
beta = 0.80, mde = 0.10, sigma  = 0.442, rho = 0.191, N = 112, n = NT = 2240
beta = 0.80, mde = 0.15, sigma  = 0.442, rho = 0.000, N = 12, n = NT = 240
beta = 0.80, mde = 0.15, sigma  = 0.442, rho = 0.144, N = 42, n = NT = 840
beta = 0.80, mde = 0.15, sigma  = 0.442, rho = 0.191, N = 50, n = NT = 1000

Two-sided Test (approximate)
beta = 0.80, mde = 0.05, sigma  = 0.442, rho = 0.000, N = 124, n = NT = 2480
beta = 0.80, mde = 0.05, sigma  = 0.442, rho = 0.144, N = 458, n = NT = 9160
beta = 0.80, mde = 0.05, sigma  = 0.442, rho = 0.191, N = 568, n = NT = 11360
beta = 0.80, mde = 0.10, sigma  = 0.44

Two evaluate the two-sided approximate calculations I use the statsmodels library to perform "exact" numerical calculations. This also shows how an intra-cluster corrected power calculation can be teased out of a basic power calculation two (see how I pass parameters into the solve_power() function below. For the examples considered here the approximate calculation is quite good. Identical answers are given up to the reported precision.

In [8]:
import statsmodels.stats.power as smp

print("")                
print("Two-sided Test (numerical calculation)")
for beta in power:
    for theta in mde:
        for p in rho:
            N1 = smp.NormalIndPower().solve_power(effect_size=theta/(np.sqrt((p + (1-p)/T))*sigma), nobs1 = None, \
                                           alpha = alpha, power = beta, ratio = (1-lmbda)/lmbda, \
                                           alternative='two-sided') 
            N  = np.ceil(((1-lmbda)/lmbda)*N1) + np.ceil(N1)          
            print("beta = "  + "%0.2f" % beta + ", mde = " + "%0.2f" % theta + ", sigma  = " + "%0.3f"  % sigma  \
                + ", rho = " + "%0.3f" % p    + ", N = "   + "%0.0f" % N     + ", n = NT = " + "%0.0f" % (N*T)) 


Two-sided Test (numerical calculation)
beta = 0.80, mde = 0.05, sigma  = 0.442, rho = 0.000, N = 124, n = NT = 2480
beta = 0.80, mde = 0.05, sigma  = 0.442, rho = 0.144, N = 458, n = NT = 9160
beta = 0.80, mde = 0.05, sigma  = 0.442, rho = 0.191, N = 568, n = NT = 11360
beta = 0.80, mde = 0.10, sigma  = 0.442, rho = 0.000, N = 32, n = NT = 640
beta = 0.80, mde = 0.10, sigma  = 0.442, rho = 0.144, N = 116, n = NT = 2320
beta = 0.80, mde = 0.10, sigma  = 0.442, rho = 0.191, N = 142, n = NT = 2840
beta = 0.80, mde = 0.15, sigma  = 0.442, rho = 0.000, N = 14, n = NT = 280
beta = 0.80, mde = 0.15, sigma  = 0.442, rho = 0.144, N = 52, n = NT = 1040
beta = 0.80, mde = 0.15, sigma  = 0.442, rho = 0.191, N = 64, n = NT = 1280


Thats it! Hopefully this notebook has convinced you that some decent baseline survey data and a few calculations can help you in designing a well powered program evaluation.

In [9]:
# This imports an attractive notebook style from Github
from IPython.display import HTML
from urllib.request import urlopen
html = urlopen('http://bit.ly/1Bf5Hft')
HTML(html.read().decode('utf-8'))